## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [1]:
#!pip install -U hopsworks --quiet

In [2]:
#!pip install kaggle --quiet

In [1]:
import numpy as np
import pandas as pd
import gdown

In [2]:
# install the 'sml' python module from this project in editable mode
!cd .. & pip install -e .

Obtaining file:///Users/lassehylleberg/Desktop/Data-Engineering-MLOps-Exam-Assignment/Data-Engineering-MLOps-Exam-Assignment
  Preparing metadata (setup.py) ... done
  Attempting uninstall: sml
    Found existing installation: sml 0.1
    Uninstalling sml-0.1:
      Successfully uninstalled sml-0.1
  Running setup.py develop for sml


In [3]:
# URL from Google Drive shareable link
url = 'https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk'

# Filename
output = 'ratebeer.txt'

# Download the file from the URL
gdown.download(url, output, quiet=False)

# Define the file path to the downloaded file
file_path = 'ratebeer.txt'

Downloading...
From (original): https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk
From (redirected): https://drive.google.com/uc?id=1LeoFhTNUZiZ9sbn31jZA6DGN3p36Dsvk&confirm=t&uuid=33e95e00-e113-499c-b8f5-97551870f7f5
To: /Users/lassehylleberg/Desktop/Data-Engineering-MLOps-Exam-Assignment/Data-Engineering-MLOps-Exam-Assignment/ratebeer.txt
100%|██████████| 1.74G/1.74G [00:28<00:00, 61.4MB/s]


In [4]:
# Define columns
columns = ['beer/name', 'beer/beerId', 'beer/brewerId', 'beer/ABV', 'beer/style',
           'review/appearance', 'review/aroma', 'review/palate', 'review/taste',
           'review/overall', 'review/time', 'review/profileName', 'review/text']
data = {col: [] for col in columns}

# Read the file
with open(file_path, "r", encoding="ISO-8859-1") as file:
    current_review = {}
    for line in file:
        line = line.strip()
        if line:
            try:
                key, value = line.split(": ", 1)
                current_review[key] = value
            except ValueError:
                continue
        else:
            if current_review:
                for col in columns:
                    data[col].append(current_review.get(col, np.nan))
                current_review = {}

# Check for any remaining entries to add
if current_review:
    for col in columns:
        data[col].append(current_review.get(col, np.nan))

# Create a DataFrame from the data
df = pd.DataFrame(data)

In [5]:
df.head()

,beer/name,beer/beerId,beer/brewerId,beer/ABV,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/overall,review/time,review/profileName,review/text
0,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,3/5,6/10,13/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836,8481,5.4,India Pale Ale &#40;IPA&#41;,4/5,6/10,4/5,7/10,13/20,1157241600,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716,8481,5,Bohemian Pilsener,4/5,5/10,3/5,6/10,14/20,958694400,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne..."
3,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,2/5,4/10,2/5,4/10,8/20,1157587200,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125,8481,5.4,Kölsch,2/5,4/10,2/5,4/10,8/20,1157587200,hopdog,"On tap at the Springfield, PA location. Poured..."


In [10]:
# Rename columns for Hopsworks compatibility
df.rename(columns={'beer/beerId': 'beer/beerid', 'beer/brewerId': 'beer/brewerid', 'beer/ABV': 'beer/abv', 'review/profileName': 'review/profilename'}, inplace=True)

In [11]:
# Replace / with _ in column names for Hopsworks compatibility
new_columns = {col: col.replace('/', '_').replace('beer/', 'beer_') for col in df.columns}
df.rename(columns=new_columns, inplace=True)

## <span style="color:#ff5f27;"> 🛠️ Feature Engineering </span>

In [8]:
# Function to convert fraction strings to float
def convert_fraction_to_float(x):
    try:
        num, denom = x.split('/')
        return float(num) / float(denom)
    except:
        return np.nan

# Columns that contain fraction ratings
rating_cols = ['review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'review_overall']
df[rating_cols] = df[rating_cols].applymap(convert_fraction_to_float)

# Convert numeric columns to float
numeric_cols = ['beer_abv', 'review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'review_overall']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Convert ID columns to integers, use pd.to_numeric for coercion
id_cols = ['beer_beerid', 'beer_brewerid']
df[id_cols] = df[id_cols].apply(pd.to_numeric, errors='coerce', downcast='integer')

# Convert review time from Unix time to datetime
df['review_time'] = pd.to_datetime(df['review_time'], unit='s', errors='coerce')

# Check the changes to verify that the types are as expected
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924163 entries, 0 to 2924162
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   beer_name           object        
 1   beer_beerid         float64       
 2   beer_brewerid       int16         
 3   beer_abv            float64       
 4   beer_style          object        
 5   review_appearance   float64       
 6   review_aroma        float64       
 7   review_palate       float64       
 8   review_taste        float64       
 9   review_overall      float64       
 10  review_time         datetime64[ns]
 11  review_profilename  object        
 12  review_text         object        
dtypes: datetime64[ns](1), float64(7), int16(1), object(4)
memory usage: 273.3+ MB


In [9]:
df.head()

,beer_name,beer_beerid,beer_brewerid,beer_abv,beer_style,review_appearance,review_aroma,review_palate,review_taste,review_overall,review_time,review_profilename,review_text
0,John Harvards Simcoe IPA,63836.0,8481,5.4,India Pale Ale &#40;IPA&#41;,0.8,0.6,0.6,0.6,0.65,2006-09-07,hopdog,"On tap at the Springfield, PA location. Poured..."
1,John Harvards Simcoe IPA,63836.0,8481,5.4,India Pale Ale &#40;IPA&#41;,0.8,0.6,0.8,0.7,0.65,2006-09-03,TomDecapolis,On tap at the John Harvards in Springfield PA....
2,John Harvards Cristal Pilsner,71716.0,8481,5.0,Bohemian Pilsener,0.8,0.5,0.6,0.6,0.70,2000-05-19,PhillyBeer2112,"UPDATED: FEB 19, 2003 Springfield, PA. I've ne..."
3,John Harvards Fancy Lawnmower Beer,64125.0,8481,5.4,Kölsch,0.4,0.4,0.4,0.4,0.40,2006-09-07,TomDecapolis,On tap the Springfield PA location billed as t...
4,John Harvards Fancy Lawnmower Beer,64125.0,8481,5.4,Kölsch,0.4,0.4,0.4,0.4,0.40,2006-09-07,hopdog,"On tap at the Springfield, PA location. Poured..."


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups in Hopsworks </span>

In [8]:
import hopsworks as hs
project = hs.login()
fs = project.get_feature_store()

/Users/lassehylleberg/anaconda3/envs/MLOPS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550037
2024-05-03 12:31:03,271 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.


In [14]:
# Ensure all ID columns are integers
df['beer_beerid'] = pd.to_numeric(df['beer_beerid'], errors='coerce').astype('Int64')

# Drop any rows where the primary key is NaN
beer_features = df[['beer_beerid', 'beer_name', 'beer_brewerid', 'beer_abv', 'beer_style']].drop_duplicates(subset=['beer_beerid'])
beer_features = beer_features.dropna(subset=['beer_beerid'])

# Feature group for Beers
beer_fg = fs.get_or_create_feature_group(
    name="beer_features",
    version=1,
    description="Basic information about beers",
    primary_key=['beer_beerid'],
    online_enabled=True
)

# Inserting the data
beer_fg.insert(beer_features)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550037/fs/545860/fg/772468


Uploading Dataframe: 100.00% |██████████| Rows 110364/110364 | Elapsed Time: 00:15 | Remaining Time: 00:00


Launching job: beer_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550037/jobs/named/beer_features_1_offline_fg_materialization/executions


In [17]:
# update beer feature descriptions
beer_feature_descriptions = {
    "beer_beerid": "Unique identifier for each beer.",
    "beer_name": "Name of the beer.",
    "beer_brewerid": "Identifier for the brewer of the beer.",
    "beer_abv": "Alcohol by volume percentage of the beer.",
    "beer_style": "Style or category of the beer."
}

# Update descriptions for Beer Features
for feature, description in beer_feature_descriptions.items():
    beer_fg.update_feature_description(feature, description)


In [15]:
# Feature Group for Reviews
review_features = df[['review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'review_overall', 'beer_beerid']]
review_fg = fs.get_or_create_feature_group(
    name="review_features",
    version=1,
    description="Metrics about beer reviews",
    primary_key=['beer_beerid'],
    online_enabled=True
)
review_fg.insert(review_features)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550037/fs/545860/fg/774499


Uploading Dataframe: 100.00% |██████████| Rows 2924163/2924163 | Elapsed Time: 03:32 | Remaining Time: 00:00


Launching job: review_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550037/jobs/named/review_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1d5843b90>, None)

In [18]:
# update review feature descriptions
review_feature_descriptions = {
    "review_appearance": "Rating of the beer's appearance (1-5).",
    "review_aroma": "Rating of the beer's aroma (1-5).",
    "review_palate": "Rating of the beer's palate (1-5).",
    "review_taste": "Rating of the beer's taste (1-5).",
    "review_overall": "Overall rating of the beer (1-5).",
    "beer_beerid": "Associated unique identifier for each beer."
}

# Update descriptions for Review Features
for feature, description in review_feature_descriptions.items():
    review_fg.update_feature_description(feature, description)

In [16]:
# Feature Group for reviewer profiles
reviewer_profile_features = df[['review_profilename']].drop_duplicates()
reviewer_fg = fs.get_or_create_feature_group(
    name="reviewer_profile_features",
    version=1,
    description="Profile information of reviewers",
    primary_key=['review_profilename'],
    online_enabled=True
)
reviewer_fg.insert(reviewer_profile_features)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/550037/fs/545860/fg/775517


Uploading Dataframe: 100.00% |██████████| Rows 29265/29265 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: reviewer_profile_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550037/jobs/named/reviewer_profile_features_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1d5841520>, None)

In [19]:
# update reviewer profile feature descriptions
reviewer_profile_feature_descriptions = {
    "review_profilename": "Unique username of the reviewer."
}

# Update descriptions for Reviewer Profile Features
for feature, description in reviewer_profile_feature_descriptions.items():
    reviewer_fg.update_feature_description(feature, description)